# Human Activity Recognition

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [4]:
'''

# Code to read csv file into Colaboratory from google drive:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

'''

'\n\n# Code to read csv file into Colaboratory from google drive:\n\n!pip install -U -q PyDrive\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# Authenticate and create the PyDrive client.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n'

In [0]:
'''
link = 'https://drive.google.com/open?id=1Kg_2TU9eHe-oxO3vZdz-d5QFUx3vcTS4' #X_train
link1 = 'https://drive.google.com/open?id=12S6ntD_pIBvPNVsAlxmSDeEbvi2YGnrd' # X_test
link2 = 'https://drive.google.com/open?id=1IuMv7H7XPxp7JrGixKW4JcXJ3PLvMGh6' #y_train
link3 = 'https://drive.google.com/open?id=1oDp9e3Pf1kmCnfgAzL0xTQgCsPw8nB_B' #y_test

'''

In [17]:
#fluff, id = link3.split('=')

#print (id)

1oDp9e3Pf1kmCnfgAzL0xTQgCsPw8nB_B


In [0]:
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('X_train.txt')

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [52]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [13]:
# Initializing parameters
epochs = 20
batch_size = 32
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [64]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\Shamim Ahmed\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\Shamim Ahmed\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [65]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


#### Defining the Architecture of LSTM

In [17]:
# Initialization
model = Sequential()
# configuring with parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
#Adding a dropout layer
model.add(Dropout(0.5))
#Adding dense layer
model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 48s 6ms/step - loss: 1.3960 - acc: 0.3681 - val_loss: 1.2606 - val_acc: 0.4337
Epoch 2/30
7352/7352 [==============================] - 45s 6ms/step - loss: 1.1265 - acc: 0.4986 - val_loss: 1.1264 - val_acc: 0.4727
Epoch 3/30
7352/7352 [==============================] - 85s 12ms/step - loss: 1.0019 - acc: 0.5449 - val_loss: 0.9544 - val_acc: 0.5741
Epoch 4/30
7352/7352 [==============================] - 124s 17ms/step - loss: 0.8440 - acc: 0.6209 - val_loss: 0.8784 - val_acc: 0.5914
Epoch 5/30
7352/7352 [==============================] - 124s 17ms/step - loss: 0.7386 - acc: 0.6481 - val_loss: 0.8611 - val_acc: 0.5935
Epoch 6/30
7352/7352 [==============================] - 124s 17ms/step - loss: 0.7624 - acc: 0.6353 - val_loss: 0.9017 - val_acc: 0.5864
Epoch 7/30
7352/7352 [==============================] - 127s 17ms/step - loss: 0.6645 - acc: 0.6640 - val_loss: 0.6681 - val_acc: 0.616

In [22]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 5s 2ms/step


[0.39807603972119654, 0.8903970139124533]

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  2      377       108        0                   3   
STANDING                 0       84       448        0                   0   
WALKING                  0        0         0      450                  10   
WALKING_DOWNSTAIRS       0        0         0        5                 406   
WALKING_UPSTAIRS         0        0         0       25                  13   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           36  
WALKING_DOWNSTAIRS                 9  
WALKING_UPSTAIRS                 433  


## Using LSTM units and adam optimizer:

In [18]:
# Initialization
model = Sequential()
# configuring with parameters
model.add(LSTM(units=50, input_shape=(timesteps, input_dim)))
#Adding a dropout layer
model.add(Dropout(0.6))
#Adding dense layer
model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                12000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 306       
Total params: 12,306
Trainable params: 12,306
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2630 - acc: 0.4274 - val_loss: 1.4146 - val_acc: 0.2582
Epoch 2/20
7352/7352 [==============================] - 28s 4ms/step - loss: 1.2912 - acc: 0.3980 - val_loss: 1.3180 - val_acc: 0.4628
Epoch 3/20
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2529 - acc: 0.4282 - val_loss: 1.3501 - val_acc: 0.4201
Epoch 4/20
7352/7352 [==============================] - 26s 4ms/step - loss: 1.3423 - acc: 0.4321 - val_loss: 1.2126 - val_acc: 0.5497
Epoch 5/20
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2102 - acc: 0.4778 - val_loss: 1.1773 - val_acc: 0.4805
Epoch 6/20
7352/7352 [==============================] - 29s 4ms/step - loss: 1.1754 - acc: 0.4799 - val_loss: 1.3501 - val_acc: 0.3627
Epoch 7/20
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2842 - acc: 0.4361 - val_loss: 1.2743 - val_acc: 0.5076
Epoch 8

In [25]:
# Initialization
model = Sequential()
# configuring with parameters
model.add(LSTM(units=32, input_shape=(timesteps, input_dim)))
#Adding a dropout layer
model.add(Dropout(0.4))
#Adding dense layer
model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 24s 3ms/step - loss: 1.4281 - acc: 0.4030 - val_loss: 1.2541 - val_acc: 0.4581
Epoch 2/20
7352/7352 [==============================] - 23s 3ms/step - loss: 1.2154 - acc: 0.4491 - val_loss: 1.2486 - val_acc: 0.4818
Epoch 3/20
7352/7352 [==============================] - 23s 3ms/step - loss: 1.2854 - acc: 0.4162 - val_loss: 1.3135 - val_acc: 0.3950
Epoch 4/20
7352/7352 [==============================] - 24s 3ms/step - loss: 1.1853 - acc: 0.4716 - val_loss: 1.4516 - val_acc: 0.3437
Epoch 5/20
7352/7352 [==============================] - 22s 3ms/step - loss: 1.2143 - acc: 0.4548 - val_loss: 1.4385 - val_acc: 0.3933
Epoch 6/20
7352/7352 [==============================] - 24s 3ms/step - loss: 1.2688 - acc: 0.4211 - val_loss: 1.1763 - val_acc: 0.5070
Epoch 7/20
7352/7352 [==============================] - 24s 3ms/step - loss: 1.0020 - acc: 0.5256 - val_loss: 1.0492 - val_acc: 0.5059
Epoch 8

In [26]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [28]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=30)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2323 - acc: 0.9214 - val_loss: 0.5842 - val_acc: 0.8605
Epoch 2/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2577 - acc: 0.9227 - val_loss: 0.5204 - val_acc: 0.8690
Epoch 3/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2235 - acc: 0.9320 - val_loss: 0.6263 - val_acc: 0.8626
Epoch 4/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.1926 - acc: 0.9361 - val_loss: 0.4435 - val_acc: 0.8812
Epoch 5/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.1970 - acc: 0.9339 - val_loss: 0.5430 - val_acc: 0.8700
Epoch 6/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2107 - acc: 0.9336 - val_loss: 0.6455 - val_acc: 0.8622
Epoch 7/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2114 - acc: 0.9389 - val_loss: 0.5947 - val_acc: 0.8772
Epoch 8

# Dropout = 60% 

In [30]:
# Initialization
model = Sequential()
# configuring with parameters
model.add(LSTM(units=32, input_shape=(timesteps, input_dim)))
#Adding a dropout layer
model.add(Dropout(0.6))
#Adding dense layer
model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [32]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=30)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2811 - acc: 0.9245 - val_loss: 0.4400 - val_acc: 0.8829
Epoch 2/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2721 - acc: 0.9244 - val_loss: 0.6199 - val_acc: 0.8483
Epoch 3/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2584 - acc: 0.9260 - val_loss: 0.4323 - val_acc: 0.8758
Epoch 4/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.4427 - acc: 0.8932 - val_loss: 0.4200 - val_acc: 0.8823
Epoch 5/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2506 - acc: 0.9298 - val_loss: 0.4346 - val_acc: 0.8711
Epoch 6/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2438 - acc: 0.9298 - val_loss: 0.4508 - val_acc: 0.8799
Epoch 7/30
7352/7352 [==============================] - 22s 3ms/step - loss: 0.2356 - acc: 0.9314 - val_loss: 0.4223 - val_acc: 0.8802
Epoch 8

## 2 Layer LSTM:

In [62]:
# Initialization
model = Sequential()
model.add(LSTM(units=32, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(units= 32))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_22 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_23 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [63]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=30)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 57s 8ms/step - loss: 1.3566 - acc: 0.4430 - val_loss: 1.1989 - val_acc: 0.4503
Epoch 2/30
7352/7352 [==============================] - 51s 7ms/step - loss: 1.0793 - acc: 0.5257 - val_loss: 1.2094 - val_acc: 0.5046
Epoch 3/30
7352/7352 [==============================] - 51s 7ms/step - loss: 0.9271 - acc: 0.6281 - val_loss: 0.8360 - val_acc: 0.6973
Epoch 4/30
7352/7352 [==============================] - 51s 7ms/step - loss: 0.7318 - acc: 0.7189 - val_loss: 0.6465 - val_acc: 0.7340
Epoch 5/30
7352/7352 [==============================] - 52s 7ms/step - loss: 0.6495 - acc: 0.7421 - val_loss: 0.6039 - val_acc: 0.7652
Epoch 6/30
7352/7352 [==============================] - 51s 7ms/step - loss: 0.5609 - acc: 0.7637 - val_loss: 0.6557 - val_acc: 0.7503
Epoch 7/30
7352/7352 [==============================] - 136s 19ms/step - loss: 0.5716 - acc: 0.7567 - val_loss: 0.5240 - val_acc: 0.7689
Epoch

## 2 Layer LSTM with 70% Dropout:

In [69]:
# Initialization
model = Sequential()
model.add(LSTM(units=32, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(Dropout(0.7))

model.add(LSTM(units= 32))
model.add(Dropout(0.7))

model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_50 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_26 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
lstm_51 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_27 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [71]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=40)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7352/7352 [==============================] - 54s 7ms/step - loss: 1.2440 - acc: 0.5027 - val_loss: 1.0186 - val_acc: 0.5382
Epoch 2/40
7352/7352 [==============================] - 65s 9ms/step - loss: 0.9628 - acc: 0.5875 - val_loss: 0.8396 - val_acc: 0.6308
Epoch 3/40
7352/7352 [==============================] - 64s 9ms/step - loss: 0.8538 - acc: 0.6249 - val_loss: 0.7622 - val_acc: 0.6481
Epoch 4/40
7352/7352 [==============================] - 65s 9ms/step - loss: 0.7820 - acc: 0.6579 - val_loss: 0.6884 - val_acc: 0.6498
Epoch 5/40
7352/7352 [==============================] - 66s 9ms/step - loss: 0.7334 - acc: 0.6766 - val_loss: 0.7790 - val_acc: 0.6630
Epoch 6/40
7352/7352 [==============================] - 72s 10ms/step - loss: 0.6670 - acc: 0.7101 - val_loss: 0.6182 - val_acc: 0.7296
Epoch 7/40
7352/7352 [==============================] - 67s 9ms/step - loss: 0.6133 - acc: 0.7397 - val_loss: 0.6160 - val_acc: 0.7180
Epoch 

## 2 layer LSTM with 64 units and 70 % Dropout:

In [72]:
# Initialization
model = Sequential()
model.add(LSTM(units=64, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(Dropout(0.7))

model.add(LSTM(units= 32))
model.add(Dropout(0.7))

model.add(Dense(n_classes, activation = 'sigmoid'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 128, 64)           18944     
_________________________________________________________________
dropout_28 (Dropout)         (None, 128, 64)           0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_29 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 198       
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=30)
score = model.evaluate(X_test, Y_test)
print(score)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 78s 11ms/step - loss: 1.3609 - acc: 0.4445 - val_loss: 1.1298 - val_acc: 0.4751
Epoch 2/30
7352/7352 [==============================] - 69s 9ms/step - loss: 1.0889 - acc: 0.5419 - val_loss: 0.9166 - val_acc: 0.5599
Epoch 3/30
7352/7352 [==============================] - 77s 10ms/step - loss: 0.9179 - acc: 0.6038 - val_loss: 0.8269 - val_acc: 0.5925
Epoch 4/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.9177 - acc: 0.5967 - val_loss: 1.1375 - val_acc: 0.5053
Epoch 5/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7997 - acc: 0.6332 - val_loss: 0.8154 - val_acc: 0.6125
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7841 - acc: 0.6427 - val_loss: 0.7719 - val_acc: 0.6132
Epoch 7/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7570 - acc: 0.6423 - val_loss: 0.7720 - val_acc: 0.6033
Epoch

## Conclusion:

In [ ]:
The best accuracy that we could get using LSTM model is 91.6 %. We got this using 2 layered LSTM with 